# Preprocessing and run hSBM

In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,scanpy -a Filippo_Valle -g -r -b -w

Author: Filippo_Valle

Last updated: Wed Oct 19 2022

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.5.0

pandas: 1.1.5
numpy : 1.22.4
scanpy: 1.9.1

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.19.0-21-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: 6af288ff4fa34a7f9f04237fa48bed0dbc8ec340

Git repo: git@github.com:fvalle1/topics.git

Git branch: HEAD

Watermark: 2.3.1



In [ ]:
# import some libraries
import pandas as pd
import numpy as np
import os, sys

# Download data or get the data

## Use already downloaded from GTEx v8

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', skiprows=2, sep='\t', index_col=0)
df.index=[g[:15] for g in df.index]
df=df[df.index.isin(pd.read_csv("http://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel())]
df_files=pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t', index_col=0)
df_files = df_files[df_files.index.isin(df.columns)]
df_files.head()

In [ ]:
df_files.to_csv("files.dat")

We ensure to have metadata for all files

In [ ]:
df = df[df.columns[df.columns.isin(df_files.index)]]
df.head(2)

### Sample columns
We pick some samples at random. We get $100$ samples from 10 most represented tissues

In [ ]:
rs = np.random.RandomState(seed=314)
samples = pd.DataFrame(columns=df_files.columns)
for site in df_files.groupby('SMTS').count().sort_values('SMTSD',ascending=False).index[:10]:
    samples = samples.append(df_files[df_files['SMTS']==site].sample(100, random_state=rs))

In [ ]:
# chack and verify the sampling process
samples.groupby('SMTS').count().sort_values('SMTSD',ascending=False)

In [ ]:
df[samples.index].to_csv("mainTable_all.csv")

In [ ]:
df_files.to_csv("files.dat")

# Filter genes

## Highly Variable Genes

We use scanpy to select $3000$ hvg

In [ ]:
samples = samples
genes = df.index
print(len(samples), len(genes))

In [ ]:
adata = sc.AnnData(X=df.reindex(index=genes, columns=samples.index).transpose(), obs=samples)

In [ ]:
sc.pp.log1p(adata, copy=False)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, n_bins=50)

In [ ]:
sc.pl.highly_variable_genes(adata, log=False, save='hvg.pdf')

In [ ]:
hvg = adata.var[adata.var['highly_variable']==True].index
samples = adata.obs.index

Save a new table with only hvg

In [ ]:
df.reindex(index=hvg, columns=samples).to_csv("mainTable_hv.csv")

## Select HouseKeeping

We download [Human House Keeping genes](https://www.cell.com/trends/genetics/fulltext/S0168-9525(13)00089-9?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS0168952513000899%3Fshowall%3Dtrue) from [https://www.tau.ac.il/~elieis/HKG/](https://www.tau.ac.il/~elieis/HKG/HK_exons.xlsx)

In [ ]:
!wget https://www.tau.ac.il/~elieis/HKG/HK_exons.xlsx
hk = pd.read_excel("HK_exons.xlsx")["Gene Name"].unique()

Here we use [https://www.genenames.org](https://www.genenames.org) to convert  genes' names to Ensamble' ids

In [ ]:
hgcn_url = "https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=md_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_name&format=text&submit=submit"
df_conversion=pd.read_csv(hgcn_url, sep="\t")

In [ ]:
df_hk_ensg = df_conversion[(df_conversion["Approved symbol"].isin(hk))&(df_conversion["Ensembl ID(supplied by Ensembl)"].isin(df.index))]["Ensembl ID(supplied by Ensembl)"].drop_duplicates().values

In [ ]:
df_hk = df.reindex(index=df_hk_ensg)
df_hk.to_csv("mainTable_hk.csv", index=True)

# Run hierarchical Stochastic Block Model
We run [stochastic block model](https://github.com/martingerlach/hSBM_Topicmodel/tree/develop) forked from [https://github.com/martingerlach/hSBM_Topicmodel](https://github.com/martingerlach/hSBM_Topicmodel).

Clone the repository with the code and add it to the system path

```bash
git clone --branch develop_share https://github.com/fvalle1/hSBM_Topicmodel/
```

In [ ]:
sys.path.append("/home/jovyan/work/topics/hSBM_Topicmodel/")
from sbmtm import sbmtm

# create the model
model = sbmtm()

In [ ]:
samples = samples
genes = hvg
print(len(samples), len(genes))

## Put the data into the model

You can choose to:
- **create** a model from raw data
- **load** a network provided in the form of a *graph.xml.gz* file
- load a **pretrained** model stored in a *topsbm.pkl* file

### make a graph with the data

Skip this if you want to use a *graph.xml.gz* file already provided

In [ ]:
df = pd.read_csv("mainTable_hv.csv", index_col=0)
genes = df.index
samples = pd.Series(index=df.columns)
print(len(samples), len(genes))

In [ ]:
model.make_graph_from_BoW_df(df.reindex(index=genes, columns=samples.index).dropna())

## Apply a log2 transformation
#model.make_graph_from_BoW_df(df.reindex(index=genes, columns=samples.index).dropna().applymap(lambda tpm: np.log2(tpm+1)))

## Apply a log10 transformation
#model.make_graph_from_BoW_df(df.reindex(index=genes, columns=samples.index).dropna().applymap(lambda tpm: np.log10(tpm+1)))

model.save_graph("graph.xml.gz")
model.g

### load data
Use this only if you **already have a *graph.xml.gz*** file

In [ ]:
# load graph
model.load_graph("graph.xml.gz")
model.g

### load pretrained
Use this only if you **already have a *topsbm.pkl*** file with a trained model 

In [ ]:
try:
    import graph_tool as gt
except:
    raise ImportError("please use python 3.6 kernel")

In [ ]:
# load pretrainded
import pickle

with open("topsbm/topsbm.pkl", "rb") as file:
    model = pickle.load(file)

model.g

## Run the model

This may take a long time and a lot of memory, wait before running in a local machine

In [ ]:
config = "gtex"
os.system(f"mkdir -p {config} && mkdir -p {config}/topsbm")
os.chdir(f"{config}/topsbm")

model.fit(n_init=10, parallel=True, verbose=True)
model.save_data()

# Check models
You can use these functions to inspect saved models.

Do not use unless needed

In [ ]:
pd.read_csv("mainTable.csv", index_col=0).applymap(lambda tpm: np.log10(tpm+1)).max().hist()

In [ ]:
pd.read_csv("mainTable_log.csv", index_col=0).max().hist()

In [ ]:
os.chdir("/content/drive/My Drive/phd/TOPSBM_TEST")
os.getcwd()

In [ ]:
import graph_tool as gt
import seaborn as sns
from sbmtm import sbmtm

In [ ]:
model = sbmtm()

In [ ]:
import matplotlib.pyplot as plt
def load_and_print(graph="graph.xml.gz", **kwargs):
    model.load_graph(graph)
    print(model.g)
    print(len(model.words),len(model.documents))
    data = gt.spectral.adjacency(model.g, weight=model.g.edge_properties["count"]).toarray()
    n_doc = len(model.documents)
    data = data[n_doc:,:n_doc]
    ax = sns.heatmap(data, **kwargs)
    ax.set_ylabel("words", fontsize=35, rotation=90)
    ax.yaxis.tick_left()
    ax.yaxis.set_label_position("left")

    ax.set_xlabel("documents",fontsize=35)
    ax.tick_params(labelsize=25)
    return model, data

def load_trained_and_print(graph="topsbm.pkl", **kwargs):
    import pickle
    with open(graph,"rb") as io:
    model = pickle.load(io)
    print(model.g)
    print(len(model.words),len(model.documents))
    data = gt.spectral.adjacency(model.g, weight=model.g.edge_properties["count"]).toarray()
    n_doc = len(model.documents)
    data = data[n_doc:,:n_doc]
    ax = sns.heatmap(data, **kwargs)
    ax.set_ylabel("words", fontsize=35, rotation=90)
    ax.yaxis.tick_left()
    ax.yaxis.set_label_position("left")

    ax.set_xlabel("documents",fontsize=35)
    ax.tick_params(labelsize=25)
    return model, data

In [ ]:
load_and_print("graph_log.xml.gz");

In [ ]:
load_and_print("graph_hk.xml.gz", vmax=5e3);

In [ ]:
load_and_print("graph_log10.xml.gz");